In [37]:
import torch
from torch import nn, optim
from torch.nn import functional as F

from torchvision import models

In [5]:
class LabelSmoothingLoss(nn.Module): 
    def __init__(self, classes, smoothing=0.2, dim=-1): 
        super(LabelSmoothingLoss, self).__init__() 
        self.confidence = 1.0 - smoothing
        self.smoothing = smoothing 
        self.cls = classes 
        self.dim = dim 

    def forward(self, pred, target): 
        pred = pred.log_softmax(dim=self.dim) 
        with torch.no_grad(): # true_dist = pred.data.clone() 
            true_dist = torch.zeros_like(pred) 
            true_dist.fill_(self.smoothing / (self.cls - 1)) 
            true_dist.scatter_(1, target.data.unsqueeze(1), self.confidence) 
            return torch.mean(torch.sum(-true_dist * pred, dim=self.dim))

In [49]:
ls = LabelSmoothingLoss(3)
ce = nn.CrossEntropyLoss()

In [50]:
model = models.resnet18()
model.train()
optimizer = optim.Adam(model.parameters())
sample = torch.rand(2, 3, 224, 224)

In [51]:
pred = model(sample)
label = torch.tensor([1,0])

In [52]:
loss = ce(pred, label)
loss_ls = ls(pred, label)
loss, loss_ls

(tensor(7.0793, grad_fn=<NllLossBackward>),
 tensor(707.2869, grad_fn=<MeanBackward0>))

In [55]:
class LabelSmoothingLoss(nn.Module):
    def __init__(self, classes, smoothing=0.2, dim=-1):
        super(LabelSmoothingLoss, self).__init__()
        self.confidence = 1.0 - smoothing
        self.smoothing = smoothing
        self.cls = classes
        self.dim = dim

    def forward(self, pred, target):
        pred = pred.log_softmax(dim=self.dim)
        
        with torch.no_grad():
            true_dist = torch.zeros_like(pred)
            true_dist.fill_(self.smoothing / (self.cls - 1))
            true_dist.scatter_(1, target.data.unsqueeze(1), self.confidence)

        return torch.mean(torch.sum(-true_dist * pred, dim=self.dim))

In [59]:
torch.tensor([[0.3, 0.2, 0.5],
            [0.4, 0.4, 0.2]]).softmax(dim=-1)

tensor([[0.3199, 0.2894, 0.3907],
        [0.3548, 0.3548, 0.2905]])